Here to run the code and try things 

Wiii


In [2]:
import numpy as np
import fit_shape

In [ ]:
# Here we make a fit to the 12 CO surface of the disk J1615
# input order must be: path, name, d, vlsr, z0, psi, PA, inc, ...

dicti = fit_shape.fit_Gas('/Volumes/Transcend/J1615_2020/DataWork/Results/CO/',
                    'J1615_12CO_im_uv006_', 
                    157.68, 4744.61, 0.2471, 1.1814, 325.03, 46.883, mstar=1.183, x0=0, y0=0,
                    r_min=1.5, r_max=2.3, downsample=25, clip=3, nwalkers=50, nburnin=500,nsteps=1500, beam=False, just_results=True)
                    

In [116]:
x0, y0

(0.008060039537431545, -0.007745213847745364)

In [115]:
mstar = dicti.get('mstar')
pa = dicti.get('PA')
inc = dicti.get('inc')
z0 = dicti.get('z0')
psi = dicti.get('psi')
vlsr = dicti.get('vlsr')/1000

x0 = dicti.get('x0')
y0 = dicti.get('y0')


In [32]:
mstar, pa, inc, z0, psi, vlsr =(1.2047920150052118, 325.15077530944075, 46.41703391829195, 0.2494024079038776, 1.211576566441788, 4.741979863714648)

(1.2047920150052118,
 325.15077530944075,
 46.41703391829195,
 0.2494024079038776,
 1.211576566441788,
 4.741979863714648)

In [14]:
vels_lev = np.array([3,3.7,4.4,5.1,5.8,6.5])

6

In [111]:
from Isovel import Isovel 

In [112]:
Isovel_disk = Isovel(mstar, pa, inc, z0, psi, vlsr, vels_lev)

In [113]:
Isovel

Isovel.Isovel

In [114]:
vel_thin, vel_front, vel_back = Isovel_disk.calculate_vel()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
print(vel_thin, vel_front, vel_back)

In [118]:
def isovel(phi, border, i, pa):
        # projected coordinates (should be xprime, yprime)
    nx, ny = (300,300)
    xp = np.linspace(-Rmax, Rmax, nx)
    yp = np.linspace(-Rmax, Rmax, ny)
    xxp, yyp = np.meshgrid(xp, yp)

        ###########################################################
        # Thin disk -> flat

        # deprojected coords y = yp/cosi
    xx = xxp#/np.cos(i)
    yy = yyp/np.cos(i)
    rr = np.sqrt(xx**2+yy**2)*pix_to_au # au
    rr_cm = rr*au # cm
    theta = np.arctan2(yy,xx)

        # projected line of sight velocity at x',y'
    vel_thin = np.sqrt(G*mstar/rr_cm)*np.sin(i)*np.cos(theta) # cm/s
    vel_thin /= 1e5 # km/s
    
        # rotate
    vel_thin = rotate(vel_thin,-pa, reshape=False)

        ###########################################################
        # Flared disk -> cone

        # intersection of line with cone:
        # yp = (y - t*sin(i))*cosi
        # zp = t*cos(i)
        # where t are the solution of a line intersecting a cone
        # t**2 * (np.cos(2*i) + np.cos(2*phi)) - 2*np.sin(phi)**2 * (xxp**2 + yyp**2*np.sec(i)**2 + 2*t*yyp*np.tan(i)) = 0
        # The positive and negative roots of this equation correspond tothe near and far halves
        # of the double cone, respectively

        # solve for t
    def f(t,ixp,iyp):
        return (np.cos(2*i) + np.cos(2*phi))*t**2 - (2*np.sin(phi)**2*2*iyp*np.tan(i))*t - (2*np.sin(phi)**2*(ixp**2 + iyp**2*seci**2 ))

    seci = 1/np.cos(i)
    tt_near = np.zeros(yyp.shape) # near side of the disk
    tt_far = np.zeros(yyp.shape) # far side of the disk

    for ix in range(nx):
        for iy in range(ny):
            sol = optimize.root(f, [0.], args=(xxp[ix,iy],yyp[ix,iy]), method='lm')
            if np.degrees(i)<180:
                tt_near[ix,iy] = np.abs(sol.x) # au
                tt_far[ix,iy] = -np.abs(sol.x)
            else:
                tt_far[ix,iy] = np.abs(sol.x) # au
                tt_near[ix,iy] = -np.abs(sol.x)

    xx = xxp
    yy_near = yyp/np.cos(i) + tt_near*np.sin(i)
    zz_near = tt_near*np.cos(i)
    rr_near = np.sqrt(xx**2+yy_near**2+zz_near**2)*pix_to_au # au
    #rr_pos = np.sqrt(xx**2+yy_pos**2) # au
    rr_near_cm = rr_near*au # cm
    theta_near = np.arctan2(yy_near,xx)

    yy_far = yyp/np.cos(i) + tt_far*np.sin(i)
    zz_far = tt_far*np.cos(i)
    rr_far = np.sqrt(xx**2+yy_far**2+zz_far**2)*pix_to_au # au
    #rr_neg = np.sqrt(xx**2+yy_neg**2) # au
    rr_far_cm = rr_far*au # cm
    theta_far = np.arctan2(yy_far,xx)

    vel_near = np.sqrt(G*mstar/rr_near_cm)*np.sin(i)*np.cos(theta_near)/1e5 # km/s
    vel_far = np.sqrt(G*mstar/rr_far_cm)*np.sin(i)*np.cos(theta_far)/1e5 # km/s

        # rotate
    vel_near = rotate(vel_near,pa, reshape=False)
    vel_far = rotate(vel_far,pa, reshape=False)

    
        # Plot
    mask_contour_near = rotate(rr_near, pa, reshape=False)[:][:]>border
    mask_contour_far= rotate(rr_far, pa, reshape=False)[:][:]>border
    mask_contour_thin = rotate(rr, pa, reshape=False)[:][:]>border
    ext=[0,2*Rmax,0,2*Rmax]
    #ext=[-Rmax,Rmax,-Rmax,Rmax]
    vel_near = np.ma.array(vel_near, mask=mask_contour_near)
    vel_far = np.ma.array(vel_far, mask=mask_contour_far)
    vel_thin = np.ma.array(vel_thin, mask=mask_contour_thin)

    return vel_near, vel_far, vel_thin

In [119]:
mstar, pa, inc, z0, psi, vlsr =(1.2047920150052118, 325.15077530944075, 46.41703391829195, 0.2494024079038776, 1.211576566441788, 4.741979863714648)
vels_lev = np.array([3,3.7,4.4,5.1,5.8,6.5])
pa = pa-180
phi = np.arctan(z0*1**psi)

R_max=300
v_max=5
nx=300
ny=300

pix_size = 2.9e-3
par=6.341708607533389 #mas
d = 1000/par #pc
pix_to_au = (pix_size*d)
border=500
Rmax=
inc = np.radians(inc)
pa = np.radians(pa)

vel_thin, vel_front, vel_back = isovel(phi, border, inc, pa)
print(vel_thin, vel_front, vel_back)

##################

sizefig = (10., 6.)

j_range = [30, 31, 32]
channels= np.arange(-7.99991, 16.1501, 0.35)

path = '/Volumes/Transcend/J1615_2020/DataWork/Results/CO/'
name_12CO = path+'J1615_12CO_im_uv006'
fits_image = name_12CO +'.fits'

border = 550.


image_1 = fits.open(fits_image)
data_1 = image_1[0].data[0][0]
imsize_1 = len(data_1[0])
Rmax = imsize_1/2. # pix


fig, axs = plt.subplots(3, 3, sharey=True, figsize=(16, 16))
for i in range(32,41):
    b=i-32
    a=0
    if b>2:
        a=1
        b=i-32-3
    if b>2:
        a=2
        b=i-32-6
    
    image_1 = fits.open(fits_image)
    data_1 = image_1[0].data[0][i]
    header = image_1[0].header
    
    pixscales=np.abs(3600*header['CDELT2'])
    arcs = imsize_1/2.*pixscales

    chan = i#j_range[0]
    image_12mm = np.zeros([imsize_1, imsize_1])
    image_12mm[:][:] = data_1[:][:]
    
    
    vels_lev = np.array([channels[chan]])-vlsr
    axs[a,b].imshow(image_12mm,origin='lower', cmap='gist_heat', vmin=0, vmax=0.03, extent=[arcs,-arcs,-arcs,arcs])#, norm=colors.PowerNorm(gamma=gval),vmin=rms,vmax=maxi)
    axs[a,b].contour(vel_front,vels_lev,origin='lower',colors='cyan',linestyles='dashed',linewidths=2.1, extent=[arcs,-arcs,-arcs,arcs], corner_mask=True)
    axs[a,b].contour(vel_back,vels_lev,origin='lower',colors='cyan',linestyles='dotted',linewidths=1.2, extent=[arcs,-arcs,-arcs,arcs], corner_mask=True)
    axs[a,b].set_xlim(3, -3)
    axs[a,b].set_ylim(-3, 3)

fig.savefig('J1615_isovelocities_2.png',dpi=600)

NameError: name 'Rmax' is not defined